In [1]:
# !pip install xgboost
import pandas as pd
import numpy as np
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split 
from sklearn import metrics
from xgboost import XGBRegressor

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
29420,2022-07-18 12:55:31,2022-07-18,12:55:31,12,20.14160,0.00000,0.000000,0.000000
29421,2022-07-18 12:56:31,2022-07-18,12:56:31,12,20.19043,0.00000,0.000000,0.000000
29422,2022-07-18 12:57:32,2022-07-18,12:57:32,12,16.28418,3.55189,57.839616,0.057840
29423,2022-07-18 12:58:32,2022-07-18,12:58:32,12,16.11328,3.64709,58.766582,0.058767
29424,2022-07-18 12:59:32,2022-07-18,12:59:32,12,20.11719,0.00000,0.000000,0.000000


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000', '2022-06-27T00:00:00.000000000',
       '2022-06-28T00:00:00.000000000', '2022-06-29T00:00:00.000000000',
       '2022-06-30T00:00:00.000000000', '2022-07-01T00:00:00.000000000',
       '2022-07-02T00:00:00.000000000', '2022-07-03T00:00:00.000000000',
       '2022-07-04T00:00:00.000000000', '2022-07-05T00:00:00.000000000',
       '2022-07-06T00:00:00.000000000', '2022-07-07T00:00:00.000000000',
       '2022-07-08T00:00:00.000000000', '2022-07-09T00:00:00.000000000',
       '2022-07-10T00:00:00.000000000', '2022-07-11T00:00:00.000000000',
       '2022-07-12T00:00:00.000000000', '2022-07-13T00:00:00.000000000',
       '2022-07-14T00:00:00.000000000', '2022-07-15T00:00:00.000000000',
       '2022-07-16T00:00:00.000000000', '2022-07-17T00:00:00.000000000',
       '2022-07-18T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
today_hourly_values['Power (KW)'].sum()

9.4429959368432

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
28642,2022-07-17,20,0.0
28643,2022-07-17,21,0.0
28644,2022-07-17,22,0.0
28645,2022-07-17,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.000000
1,2022-06-25,1,0.000000
2,2022-06-25,2,0.000000
3,2022-06-25,3,0.000000
4,2022-06-25,4,0.000000
...,...,...,...
547,2022-07-17,19,0.202232
548,2022-07-17,20,0.000676
549,2022-07-17,21,0.000000
550,2022-07-17,22,0.000000


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
# weather_data.drop(['Date', 'Time', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
#                    'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
# weather_data.drop(['Date', 'Time','DewPoint (°C)', 'Direction', 'Visibility (km)', 
#                    'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status', 'Hum (%)',
#                    'Temp (°C)', 'RealFeelTemp (°C)'], axis = 1, inplace = True)
weather_data.drop(['Date', 'Time','DewPoint (°C)', 'Direction', 'Visibility (km)', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status', 'Hum (%)',
                   'CloudCover (%)', 'Temp (°C)'], axis = 1, inplace = True)
weather_data.tail(14)

,SolarIrradiance (W/m2),RealFeelTemp (°C),Wind (km/h),UVIndex
562,644.8,28.7,9.3,4
563,675.2,31.5,11.1,5
564,705.5,34.7,13.0,7
565,718.8,36.6,14.8,7
566,710.0,37.0,14.8,7
567,679.2,38.0,14.8,5
568,672.9,37.4,14.8,4
569,592.6,36.7,13.0,3
570,488.6,35.7,13.0,2
571,361.1,34.1,13.0,1


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
# df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Wind (km/h)', 'UVIndex', 'CloudCover (%)']] = 0
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(25)

,Power (KW),SolarIrradiance (W/m2),RealFeelTemp (°C),Wind (km/h),UVIndex
551,0.0,0.0,0.0,0.0,0
552,NaN,0.0,0.0,0.0,0
553,NaN,0.0,0.0,0.0,0
554,NaN,0.0,0.0,0.0,0
555,NaN,0.0,0.0,0.0,0
556,NaN,0.0,0.0,0.0,0
557,NaN,0.0,0.0,0.0,0
558,NaN,138.4,15.7,7.4,1
559,NaN,272.6,19.3,7.4,1
560,NaN,445.3,21.9,7.4,2


In [12]:
count_total_rows = len(df1) - 24

In [13]:
# independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]
# independent_columns = df1[['SolarIrradiance (W/m2)', 'Wind (km/h)', 'UVIndex', 'CloudCover (%)']][0:count_total_rows]
independent_columns = df1[['SolarIrradiance (W/m2)', 'RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']][0:count_total_rows]

In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
# xgbr = XGBRegressor(n_estimators=69)
xgbr = XGBRegressor() 
xgbr.fit(independent_columns, dependent_column)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

#### Is model well fit or not?

In [16]:
r_sq = xgbr.score(independent_columns, dependent_column)
r_sq

0.9998543315317872

In [17]:
# forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)
# forcasted_data = df1[['SolarIrradiance (W/m2)', 'Wind (km/h)', 'UVIndex', 'CloudCover (%)']].tail(24)
forcasted_data = df1[['SolarIrradiance (W/m2)', 'RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']].tail(24)

In [18]:
return_array = list(xgbr.predict(forcasted_data))

In [19]:
# predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
# predicted_data

In [20]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [21]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [22]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [23]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-07-18,0,0.000023
1,2022-07-18,1,0.000023
2,2022-07-18,2,0.000023
3,2022-07-18,3,0.000023
4,2022-07-18,4,0.000023
5,2022-07-18,5,0.000023
6,2022-07-18,6,0.234136
7,2022-07-18,7,0.251742
8,2022-07-18,8,1.404905
9,2022-07-18,9,1.370045


In [24]:
data_dataframe['Power (KW)'].sum()

19.311062

## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [25]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], 
                                           data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.1304153422140158

In [26]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.3611306442466712

In [27]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], 
                                            data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.1816329964117066

#### R squared

In [28]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], 
                             data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.8357930031829609

#### Yesterday results

In [30]:
filter_last_day_values = df[df['Date'] == unique_date[-2]][['Date', 'Hour', 'Power (KW)']]
last_day_hourly_values = filter_last_day_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()

filter_yes_values = df[(df['Date'] >= '2022-06-25') & (df['Date'] <= unique_date[-3])][['Date', 'Hour', 'Power (KW)']]
yes_hourly_values = filter_yes_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data1 = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_unique_date = weather_data1['Date'].unique()
filter_weather_yes_values = weather_data1[
                                         (weather_data1['Date'] >= '2022-06-25') & 
                                         (weather_data1['Date'] <= weather_unique_date[-3])][['SolarIrradiance (W/m2)', 'RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']]
yes_df1 = pd.concat([yes_hourly_values, filter_weather_yes_values], axis = 1)
yes_df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
yes_df1.loc[yes_df1['SolarIrradiance (W/m2)'] == 0, ['RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']] = 0
yes_count_total_rows = len(yes_df1)
yes_independent_columns = yes_df1[['SolarIrradiance (W/m2)', 'RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']][0:yes_count_total_rows]
yes_dependent_column = yes_df1['Power (KW)'][0:yes_count_total_rows]
yes_reg = XGBRegressor() 
yes_reg.fit(yes_independent_columns, yes_dependent_column)
forcasted_yes_values = weather_data1[(weather_data1['Date'] == weather_unique_date[-2])][['SolarIrradiance (W/m2)', 'RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']]
forcasted_yes_values.loc[forcasted_yes_values['SolarIrradiance (W/m2)'] == 0, ['RealFeelTemp (°C)', 'Wind (km/h)', 'UVIndex']] = 0
return_array = yes_reg.predict(forcasted_yes_values)
predicted_data = pd.DataFrame(return_array, columns = ['Power (KW)'])
mv_pe = predicted_data['Power (KW)'].sum()
mv_mse = metrics.mean_squared_error(last_day_hourly_values['Power (KW)'], predicted_data['Power (KW)'])
mv_rmse = np.sqrt(mv_mse)
mv_mae = metrics.mean_absolute_error(last_day_hourly_values['Power (KW)'], predicted_data['Power (KW)'])
mv_rs = metrics.r2_score(last_day_hourly_values['Power (KW)'], predicted_data['Power (KW)'])
mv_rs

0.7877746666219256